In [7]:
# !pip install -q datasets transformers 
# !pip install --upgrade accelerate
%config Completer.use_jedi = False

In [8]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, T5ForConditionalGeneration, TrainingArguments, Trainer, BertForQuestionAnswering
from datasets import load_dataset
model_checkpoint = "t5-small"

In [9]:
squad = load_dataset("squad", split="train[:5000]")

In [10]:
squad = squad.train_test_split(test_size=0.2)

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)


    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [13]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
import torch
torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir="qa_T5",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to = 'none' 
)

model = BertForQuestionAnswering.from_pretrained(model_checkpoint)
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if training_args.fp16 else None,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

You are using a model of type t5 to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at t5-small were not used when initializing BertForQuestionAnswering: ['decoder.final_layer_norm.weight', 'encoder.block.2.layer.0.SelfAttention.v.weight', 'decoder.block.1.layer.0.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.3.layer.0.SelfAttention.o.weight', 'decoder.block.4.layer.2.DenseReluDense.wo.weight', 'decoder.block.4.layer.1.EncDecAttention.k.weight', 'decoder.block.2.layer.2.DenseReluDense.wi.weight', 'encoder.block.2.layer.0.SelfAttention.o.weight', 'decoder.block.2.layer.0.SelfAttention.o.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'encoder.block.4.layer.0.SelfAttention.o.weight', 'decoder.block.2.layer.2.layer_norm.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.4.layer.1.EncDecAttention.o.weight', 'encoder.block.

Epoch,Training Loss,Validation Loss
1,No log,4.632058
2,4.631100,4.583677
3,4.631100,4.669224


TrainOutput(global_step=750, training_loss=4.423497721354167, metrics={'train_runtime': 515.4332, 'train_samples_per_second': 23.281, 'train_steps_per_second': 1.455, 'total_flos': 2351670810624000.0, 'train_loss': 4.423497721354167, 'epoch': 3.0})